In [5]:
import pandas as pd
from io import StringIO
import requests
import re
import matplotlib.pyplot as plt
import warnings

pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.options.display.max_colwidth = None
warnings.filterwarnings("ignore")

## Atp Matches File

In [6]:
# Function to list files in a GitHub repository
def list_github_files(repo_name: str) -> list | None:
    url = f"https://api.github.com/repos/{repo_name}/contents/"
    response = requests.get(url)
    if response.status_code == 200:
        files = response.json()
        file_names = [file["name"] for file in files]
        return file_names
    else:
        print(f"Failed to retrieve files: {response.status_code}")


def read_csv_from_github(repo_name: str, file_name: str) -> pd.DataFrame:
    """
    Reads a CSV file from a GitHub repository into a Pandas DataFrame.
    Parameters:
        repo_name (str): The name of the GitHub repository (e.g., "owner/repo").
        file_name (str): The name of the CSV file.
    Returns:
        pd.DataFrame: A Pandas DataFrame containing the data from the CSV file.
    """

    file_url = f"https://raw.githubusercontent.com/{repo_name}/master/{file_name}"
    response = requests.get(file_url, verify=False)
    csv_content = response.content.decode("utf-8")
    df = pd.read_csv(StringIO(csv_content))
    return df

In [7]:
repo_name = "JeffSackmann/tennis_atp"
file_names = list_github_files(repo_name)

# Filter files to only include CSV files
file_names = [file for file in file_names if file.endswith(".csv")]

# Filter files to only include those that follow the "atp_matches_YYYY" pattern
atp_matches = [file for file in file_names if re.match(r"atp_matches_\d{4}\.csv", file)]

# Read and aggregate ATP matches data
atp_matches_dfs = [read_csv_from_github(repo_name, file) for file in atp_matches]
atp_matches_df = pd.concat(atp_matches_dfs, ignore_index=True)

# Display df info for ATP matches
print(
    "ATP Matches DataFrame Info:",
    "\nShape:",
    atp_matches_df.shape,
    "\nColumns:",
    atp_matches_df.columns,
)

ATP Matches DataFrame Info: 
Shape: (194996, 49) 
Columns: Index(['tourney_id', 'tourney_name', 'surface', 'draw_size', 'tourney_level',
       'tourney_date', 'match_num', 'winner_id', 'winner_seed', 'winner_entry',
       'winner_name', 'winner_hand', 'winner_ht', 'winner_ioc', 'winner_age',
       'loser_id', 'loser_seed', 'loser_entry', 'loser_name', 'loser_hand',
       'loser_ht', 'loser_ioc', 'loser_age', 'score', 'best_of', 'round',
       'minutes', 'w_ace', 'w_df', 'w_svpt', 'w_1stIn', 'w_1stWon', 'w_2ndWon',
       'w_SvGms', 'w_bpSaved', 'w_bpFaced', 'l_ace', 'l_df', 'l_svpt',
       'l_1stIn', 'l_1stWon', 'l_2ndWon', 'l_SvGms', 'l_bpSaved', 'l_bpFaced',
       'winner_rank', 'winner_rank_points', 'loser_rank', 'loser_rank_points'],
      dtype='object')


In [8]:
# Convert the 'tourney_date' column to datetime format
atp_matches_df["tourney_date"] = pd.to_datetime(
    atp_matches_df["tourney_date"], format="%Y%m%d"
)
# Create tournay_year column
atp_matches_df["tourney_year"] = atp_matches_df["tourney_date"].dt.year
# Filter for matches after selected year
selected_year = 1991
atp_matches_df = atp_matches_df[
    atp_matches_df["tourney_year"] >= selected_year
].reset_index(drop=True)
# Filter for tourney levels in ['G', 'F', 'M', 'A']
atp_matches_df = atp_matches_df[
    atp_matches_df["tourney_level"].isin(["G", "F", "M", "A"])
].reset_index(drop=True)
# Exclude Laver Cup matches
atp_matches_df = atp_matches_df[
    ~atp_matches_df["tourney_name"].str.contains("Laver Cup")
].reset_index(drop=True)

# Count the number of NaN values for each column in the ATP matches DataFrame
nan_counts_matches = atp_matches_df.isna().sum() / atp_matches_df.shape[0] * 100
print("Dataframe shape: ", atp_matches_df.shape)
print("NaN % in ATP matches DataFrame from {}:".format(selected_year))
print(nan_counts_matches)

Dataframe shape:  (98355, 50)
NaN % in ATP matches DataFrame from 1991:
tourney_id             0.000000
tourney_name           0.000000
surface                0.000000
draw_size              0.000000
tourney_level          0.000000
tourney_date           0.000000
match_num              0.000000
winner_id              0.000000
winner_seed           55.391185
winner_entry          86.417569
winner_name            0.000000
winner_hand            0.000000
winner_ht              0.095572
winner_ioc             0.000000
winner_age             0.000000
loser_id               0.000000
loser_seed            75.417620
loser_entry           77.741853
loser_name             0.000000
loser_hand             0.000000
loser_ht               0.721875
loser_ioc              0.000000
loser_age              0.012201
score                  0.000000
best_of                0.000000
round                  0.000000
minutes                4.663718
w_ace                  1.810787
w_df                   1.810787


In [9]:
atp_matches_df.head()

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,winner_name,winner_hand,winner_ht,winner_ioc,winner_age,loser_id,loser_seed,loser_entry,loser_name,loser_hand,loser_ht,loser_ioc,loser_age,score,best_of,round,minutes,w_ace,w_df,w_svpt,w_1stIn,w_1stWon,w_2ndWon,w_SvGms,w_bpSaved,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points,tourney_year
0,1991-301,Auckland,Hard,32.0,A,1991-01-07,1,101142,1.0,NaN,Emilio Sanchez,R,180.0,ESP,25.6,101746,NaN,NaN,Renzo Furlan,R,175.0,ITA,20.6,6-4 6-1,3,R32,63.0,1.0,0.0,53.0,37.0,30.0,7.0,9.0,5.0,6.0,3.0,0.0,46.0,30.0,17.0,7.0,8.0,2.0,6.0,9.0,1487.0,78.0,459.0,1991
1,1991-301,Auckland,Hard,32.0,A,1991-01-07,2,101613,NaN,Q,Malivai Washington,R,180.0,USA,21.5,100587,NaN,WC,Steve Guy,R,188.0,NZL,31.8,6-3 6-2,3,R32,72.0,5.0,1.0,56.0,25.0,17.0,20.0,9.0,1.0,2.0,4.0,7.0,56.0,30.0,22.0,6.0,8.0,7.0,11.0,94.0,371.0,220.0,114.0,1991
2,1991-301,Auckland,Hard,32.0,A,1991-01-07,3,101179,NaN,NaN,Jean Philippe Fleurian,R,185.0,FRA,25.3,101601,NaN,WC,Brett Steven,R,185.0,NZL,21.6,2-6 6-1 6-2,3,R32,101.0,2.0,4.0,80.0,55.0,35.0,16.0,12.0,2.0,4.0,1.0,3.0,68.0,43.0,24.0,14.0,11.0,4.0,8.0,77.0,468.0,212.0,116.0,1991
3,1991-301,Auckland,Hard,32.0,A,1991-01-07,4,101117,NaN,NaN,Eric Jelen,R,180.0,GER,25.8,101332,8.0,NaN,Gilad Bloom,L,173.0,ISR,23.8,6-3 1-6 6-4,3,R32,108.0,0.0,1.0,82.0,55.0,35.0,14.0,13.0,6.0,10.0,3.0,2.0,96.0,61.0,38.0,15.0,13.0,8.0,12.0,65.0,502.0,72.0,483.0,1991
4,1991-301,Auckland,Hard,32.0,A,1991-01-07,5,101901,NaN,Q,Chuck Adams,R,185.0,USA,19.7,101735,3.0,NaN,Richard Fromberg,R,196.0,AUS,20.6,6-3 6-4,3,R32,65.0,4.0,4.0,65.0,46.0,34.0,12.0,10.0,2.0,2.0,1.0,3.0,49.0,25.0,21.0,12.0,9.0,4.0,6.0,190.0,142.0,28.0,876.0,1991


In [10]:
# Save the file in the data folder as pickle
atp_matches_df.to_pickle("../data/matches_results.pkl")

In [11]:
atp_matches_df = pd.read_pickle("../data/matches_results.pkl")
print("Dataframe shape: ", atp_matches_df.shape)

Dataframe shape:  (98355, 50)


### save atp_matches file
todo:
1. save the file
2. create loader that checks latest files and updates if there's new files

## Player Stats Historical

In [12]:
atp_matches_df.columns

Index(['tourney_id', 'tourney_name', 'surface', 'draw_size', 'tourney_level',
       'tourney_date', 'match_num', 'winner_id', 'winner_seed', 'winner_entry',
       'winner_name', 'winner_hand', 'winner_ht', 'winner_ioc', 'winner_age',
       'loser_id', 'loser_seed', 'loser_entry', 'loser_name', 'loser_hand',
       'loser_ht', 'loser_ioc', 'loser_age', 'score', 'best_of', 'round',
       'minutes', 'w_ace', 'w_df', 'w_svpt', 'w_1stIn', 'w_1stWon', 'w_2ndWon',
       'w_SvGms', 'w_bpSaved', 'w_bpFaced', 'l_ace', 'l_df', 'l_svpt',
       'l_1stIn', 'l_1stWon', 'l_2ndWon', 'l_SvGms', 'l_bpSaved', 'l_bpFaced',
       'winner_rank', 'winner_rank_points', 'loser_rank', 'loser_rank_points',
       'tourney_year'],
      dtype='object')

In [13]:
cols = [
    "tourney_id",
    "tourney_name",
    "surface",
    "draw_size",
    "tourney_level",
    "tourney_date",
    "tourney_year",
    "match_num",  # tournament info
    "winner_id",
    "winner_seed",
    "winner_entry",
    "winner_name",
    "winner_hand",
    "winner_ht",
    "winner_ioc",
    "winner_age",
    "winner_rank",
    "winner_rank_points",  # winner info
    "w_ace",
    "w_df",
    "w_svpt",
    "w_1stIn",
    "w_1stWon",
    "w_2ndWon",
    "w_SvGms",
    "w_bpSaved",
    "w_bpFaced",  # winner stats
    "loser_id",
    "loser_seed",
    "loser_entry",
    "loser_name",
    "loser_hand",
    "loser_ht",
    "loser_ioc",
    "loser_age",
    "loser_rank",
    "loser_rank_points",  # loser info
    "l_ace",
    "l_df",
    "l_svpt",
    "l_1stIn",
    "l_1stWon",
    "l_2ndWon",
    "l_SvGms",
    "l_bpSaved",
    "l_bpFaced",
    "score",
    "best_of",
    "round",
    "minutes",  # match info
]

In [14]:
player_stats_hist_df = atp_matches_df[cols].copy()
player_stats_hist_df_l = atp_matches_df[cols].copy()

# add a column to flag winner and loser
player_stats_hist_df["results"] = 1
player_stats_hist_df_l["results"] = 0
player_stats_hist_df.head()

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,tourney_year,match_num,winner_id,winner_seed,winner_entry,winner_name,winner_hand,winner_ht,winner_ioc,winner_age,winner_rank,winner_rank_points,w_ace,w_df,w_svpt,w_1stIn,w_1stWon,w_2ndWon,w_SvGms,w_bpSaved,w_bpFaced,loser_id,loser_seed,loser_entry,loser_name,loser_hand,loser_ht,loser_ioc,loser_age,loser_rank,loser_rank_points,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,score,best_of,round,minutes,results
0,1991-301,Auckland,Hard,32.0,A,1991-01-07,1991,1,101142,1.0,NaN,Emilio Sanchez,R,180.0,ESP,25.6,9.0,1487.0,1.0,0.0,53.0,37.0,30.0,7.0,9.0,5.0,6.0,101746,NaN,NaN,Renzo Furlan,R,175.0,ITA,20.6,78.0,459.0,3.0,0.0,46.0,30.0,17.0,7.0,8.0,2.0,6.0,6-4 6-1,3,R32,63.0,1
1,1991-301,Auckland,Hard,32.0,A,1991-01-07,1991,2,101613,NaN,Q,Malivai Washington,R,180.0,USA,21.5,94.0,371.0,5.0,1.0,56.0,25.0,17.0,20.0,9.0,1.0,2.0,100587,NaN,WC,Steve Guy,R,188.0,NZL,31.8,220.0,114.0,4.0,7.0,56.0,30.0,22.0,6.0,8.0,7.0,11.0,6-3 6-2,3,R32,72.0,1
2,1991-301,Auckland,Hard,32.0,A,1991-01-07,1991,3,101179,NaN,NaN,Jean Philippe Fleurian,R,185.0,FRA,25.3,77.0,468.0,2.0,4.0,80.0,55.0,35.0,16.0,12.0,2.0,4.0,101601,NaN,WC,Brett Steven,R,185.0,NZL,21.6,212.0,116.0,1.0,3.0,68.0,43.0,24.0,14.0,11.0,4.0,8.0,2-6 6-1 6-2,3,R32,101.0,1
3,1991-301,Auckland,Hard,32.0,A,1991-01-07,1991,4,101117,NaN,NaN,Eric Jelen,R,180.0,GER,25.8,65.0,502.0,0.0,1.0,82.0,55.0,35.0,14.0,13.0,6.0,10.0,101332,8.0,NaN,Gilad Bloom,L,173.0,ISR,23.8,72.0,483.0,3.0,2.0,96.0,61.0,38.0,15.0,13.0,8.0,12.0,6-3 1-6 6-4,3,R32,108.0,1
4,1991-301,Auckland,Hard,32.0,A,1991-01-07,1991,5,101901,NaN,Q,Chuck Adams,R,185.0,USA,19.7,190.0,142.0,4.0,4.0,65.0,46.0,34.0,12.0,10.0,2.0,2.0,101735,3.0,NaN,Richard Fromberg,R,196.0,AUS,20.6,28.0,876.0,1.0,3.0,49.0,25.0,21.0,12.0,9.0,4.0,6.0,6-3 6-4,3,R32,65.0,1


In [15]:
# Rename columns to remove "winner_" prefix and "w_" prefix only if they start the column name
player_stats_hist_df.columns = [
    col.replace("winner_", "player_", 1)
    if col.startswith("winner_")
    else col.replace("w_", "p_", 1)
    if col.startswith("w_")
    else col.replace("loser_", "opponent_", 1)
    if col.startswith("loser_")
    else col.replace("l_", "o_", 1)
    if col.startswith("l_")
    else col
    for col in player_stats_hist_df.columns
]

player_stats_hist_df_l.columns = [
    col.replace("loser_", "player_", 1)
    if col.startswith("loser_")
    else col.replace("l_", "p_", 1)
    if col.startswith("l_")
    else col.replace("winner_", "opponent_", 1)
    if col.startswith("winner_")
    else col.replace("w_", "o_", 1)
    if col.startswith("w_")
    else col
    for col in player_stats_hist_df_l.columns
]

In [16]:
# Append the loser stats to the winner stats
player_stats_hist_df = pd.concat(
    [player_stats_hist_df, player_stats_hist_df_l], ignore_index=True
)
player_stats_hist_df.head()

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,tourney_year,match_num,player_id,player_seed,player_entry,player_name,player_hand,player_ht,player_ioc,player_age,player_rank,player_rank_points,p_ace,p_df,p_svpt,p_1stIn,p_1stWon,p_2ndWon,p_SvGms,p_bpSaved,p_bpFaced,opponent_id,opponent_seed,opponent_entry,opponent_name,opponent_hand,opponent_ht,opponent_ioc,opponent_age,opponent_rank,opponent_rank_points,o_ace,o_df,o_svpt,o_1stIn,o_1stWon,o_2ndWon,o_SvGms,o_bpSaved,o_bpFaced,score,best_of,round,minutes,results
0,1991-301,Auckland,Hard,32.0,A,1991-01-07,1991,1,101142,1.0,NaN,Emilio Sanchez,R,180.0,ESP,25.6,9.0,1487.0,1.0,0.0,53.0,37.0,30.0,7.0,9.0,5.0,6.0,101746,NaN,NaN,Renzo Furlan,R,175.0,ITA,20.6,78.0,459.0,3.0,0.0,46.0,30.0,17.0,7.0,8.0,2.0,6.0,6-4 6-1,3,R32,63.0,1
1,1991-301,Auckland,Hard,32.0,A,1991-01-07,1991,2,101613,NaN,Q,Malivai Washington,R,180.0,USA,21.5,94.0,371.0,5.0,1.0,56.0,25.0,17.0,20.0,9.0,1.0,2.0,100587,NaN,WC,Steve Guy,R,188.0,NZL,31.8,220.0,114.0,4.0,7.0,56.0,30.0,22.0,6.0,8.0,7.0,11.0,6-3 6-2,3,R32,72.0,1
2,1991-301,Auckland,Hard,32.0,A,1991-01-07,1991,3,101179,NaN,NaN,Jean Philippe Fleurian,R,185.0,FRA,25.3,77.0,468.0,2.0,4.0,80.0,55.0,35.0,16.0,12.0,2.0,4.0,101601,NaN,WC,Brett Steven,R,185.0,NZL,21.6,212.0,116.0,1.0,3.0,68.0,43.0,24.0,14.0,11.0,4.0,8.0,2-6 6-1 6-2,3,R32,101.0,1
3,1991-301,Auckland,Hard,32.0,A,1991-01-07,1991,4,101117,NaN,NaN,Eric Jelen,R,180.0,GER,25.8,65.0,502.0,0.0,1.0,82.0,55.0,35.0,14.0,13.0,6.0,10.0,101332,8.0,NaN,Gilad Bloom,L,173.0,ISR,23.8,72.0,483.0,3.0,2.0,96.0,61.0,38.0,15.0,13.0,8.0,12.0,6-3 1-6 6-4,3,R32,108.0,1
4,1991-301,Auckland,Hard,32.0,A,1991-01-07,1991,5,101901,NaN,Q,Chuck Adams,R,185.0,USA,19.7,190.0,142.0,4.0,4.0,65.0,46.0,34.0,12.0,10.0,2.0,2.0,101735,3.0,NaN,Richard Fromberg,R,196.0,AUS,20.6,28.0,876.0,1.0,3.0,49.0,25.0,21.0,12.0,9.0,4.0,6.0,6-3 6-4,3,R32,65.0,1


In [17]:
player_stats_hist_df.shape

(196710, 51)

In [18]:
# First ensure the DataFrame is properly sorted by the specified order
player_stats_hist_df = player_stats_hist_df.sort_values(
    by=["player_id", "tourney_date", "tourney_id", "match_num"]
).reset_index(drop=True)

player_stats_hist_df.head()

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,tourney_year,match_num,player_id,player_seed,player_entry,player_name,player_hand,player_ht,player_ioc,player_age,player_rank,player_rank_points,p_ace,p_df,p_svpt,p_1stIn,p_1stWon,p_2ndWon,p_SvGms,p_bpSaved,p_bpFaced,opponent_id,opponent_seed,opponent_entry,opponent_name,opponent_hand,opponent_ht,opponent_ioc,opponent_age,opponent_rank,opponent_rank_points,o_ace,o_df,o_svpt,o_1stIn,o_1stWon,o_2ndWon,o_SvGms,o_bpSaved,o_bpFaced,score,best_of,round,minutes,results
0,1992-409,Atlanta,Clay,32.0,A,1992-04-27,1992,12,100282,NaN,WC,Guillermo Vilas,L,180.0,ARG,39.6,444.0,38.0,0.0,2.0,111.0,94.0,55.0,9.0,17.0,2.0,8.0,101334,3.0,NaN,Alexander Volkov,L,188.0,RUS,25.1,18.0,1247.0,2.0,3.0,126.0,71.0,51.0,20.0,16.0,10.0,15.0,5-7 7-6(6) 6-3,3,R32,167.0,0
1,1992-323,Bordeaux,Clay,32.0,A,1992-09-14,1992,7,100282,NaN,WC,Guillermo Vilas,L,180.0,ARG,40.0,410.0,47.0,0.0,3.0,81.0,41.0,26.0,20.0,13.0,5.0,9.0,102001,NaN,NaN,German Lopez,R,193.0,ESP,20.7,99.0,425.0,11.0,2.0,79.0,46.0,35.0,17.0,12.0,5.0,6.0,6-4 3-6 6-0,3,R32,109.0,0
2,1991-411,Chicago,Carpet,32.0,A,1991-02-25,1991,10,100284,NaN,WC,Jimmy Connors,L,178.0,USA,38.4,990.0,2.0,0.0,1.0,41.0,33.0,13.0,3.0,7.0,2.0,7.0,101409,NaN,NaN,Jaime Yzaga,R,170.0,PER,23.3,65.0,502.0,0.0,2.0,49.0,33.0,24.0,7.0,8.0,1.0,2.0,6-3 6-0,3,R32,58.0,0
3,1991-403,Miami Masters,Hard,96.0,M,1991-03-15,1991,15,100284,NaN,WC,Jimmy Connors,L,178.0,USA,38.5,961.0,3.0,0.0,2.0,69.0,47.0,27.0,10.0,10.0,5.0,9.0,101274,NaN,NaN,Udo Riglewski,R,185.0,GER,24.6,98.0,364.0,3.0,4.0,70.0,33.0,21.0,13.0,10.0,5.0,11.0,6-4 6-4,3,R128,107.0,1
4,1991-403,Miami Masters,Hard,96.0,M,1991-03-15,1991,47,100284,NaN,WC,Jimmy Connors,L,178.0,USA,38.5,961.0,3.0,0.0,3.0,50.0,33.0,15.0,8.0,9.0,2.0,7.0,101750,28.0,NaN,Cristiano Caratti,R,178.0,ITA,20.7,37.0,767.0,3.0,3.0,66.0,43.0,25.0,12.0,10.0,3.0,6.0,6-4 6-3,3,R64,80.0,0


In [19]:
from typing import Union, List
import pandas as pd


def add_rolling_stats(
    df: pd.DataFrame,
    stats_columns: Union[str, List[str]],
    agg_type: str = "mean",
    window: int = 5,
    min_periods: int = 1,
    shift_periods: int = 1,
    group_col: str = "player_id",
    sort_cols: List[str] = ["player_id", "tourney_date", "tourney_id", "match_num"],
) -> pd.DataFrame:
    """
    Add rolling statistics to a DataFrame for each player.

    Parameters:
    -----------
    df : pd.DataFrame
        The input DataFrame
    player_id_col : str
        Column name for player identification
    sort_cols : List[str]
        Columns to sort by to ensure proper chronological order
    stats_columns : Union[str, List[str], None]
        Column(s) to calculate rolling statistics for
    agg_type : str
        Type of aggregation ('mean', 'sum', 'std', 'min', 'max', 'median')
    window : int
        Number of periods for the rolling window
    min_periods : int
        Minimum number of observations required to have a value
    shift_periods : int
        Number of periods to shift (1 = exclude current match, 0 = include current match)

    Returns:
    --------
    pd.DataFrame
        DataFrame with added rolling statistics columns
    """

    # Make a copy to avoid modifying the original DataFrame
    result_df = df.copy()

    # Ensure proper sorting
    result_df = result_df.sort_values(by=sort_cols).reset_index(drop=True)

    # Convert single column to list
    if isinstance(stats_columns, str):
        stats_columns = [stats_columns]

    # Calculate rolling statistics for each column
    for col in stats_columns:
        if col in result_df.columns:
            # Create column name for the new statistic
            new_col_name = f"{col}_{agg_type}_last{window}"

            # Calculate rolling statistic
            rolling_obj = result_df.groupby(group_col)[col].rolling(
                window=window, min_periods=min_periods
            )

            # Apply the specified aggregation
            if agg_type == "mean":
                rolling_stat = rolling_obj.mean()
            elif agg_type == "sum":
                rolling_stat = rolling_obj.sum()
            elif agg_type == "std":
                rolling_stat = rolling_obj.std()
            elif agg_type == "min":
                rolling_stat = rolling_obj.min()
            elif agg_type == "max":
                rolling_stat = rolling_obj.max()
            elif agg_type == "median":
                rolling_stat = rolling_obj.median()
            else:
                raise ValueError(f"Unsupported aggregation type: {agg_type}")

            # Apply shift and add to DataFrame
            result_df[new_col_name] = rolling_stat.shift(shift_periods).reset_index(
                level=0, drop=True
            )

            # Set first record for each player to NaN
            first_records = result_df.groupby(group_col).head(1).index
            result_df.loc[first_records, new_col_name] = pd.NA
        else:
            print(f"Warning: Column '{col}' not found in DataFrame")

    return result_df

In [20]:
# To Test the function:

# # Add rolling averages for multiple stats
# stats_columns = ["p_ace", "p_df", "p_svpt", "p_1stIn", "p_1stWon", "p_2ndWon", "results"]
# player_stats_hist_df = add_rolling_stats(
#     player_stats_hist_df, stats_columns=stats_columns, agg_type="mean", window=5
# )
#
# # Add rolling standard deviation for serve points
# player_stats_hist_df = add_rolling_stats(
#     player_stats_hist_df, stats_columns="p_svpt", agg_type="min", window=2
# )
#
# # Add rolling sum for aces over last 3 matches
# player_stats_hist_df = add_rolling_stats(
#     player_stats_hist_df, stats_columns="results", agg_type="sum", window=3
# )
#
# # Verify the results
# player_stats_hist_df[
#     [
#         "player_id",
#         "tourney_date",
#         "tourney_id",
#         "match_num",
#         "p_df",
#         "p_df_mean_last5",
#         "p_svpt",
#         "p_svpt_min_last2",
#         "results",
#         "results_sum_last3"
#     ]
# ].head(10)

In [21]:
cols_to_mean = [
    "p_ace",
    "p_df",
    "p_svpt",
    "p_1stIn",
    "p_1stWon",
    "p_2ndWon",
    "p_SvGms",
    "p_bpSaved",
    "p_bpFaced",  # player stats
    "opponent_rank_points",  # opponent info
    "o_ace",
    "o_df",
    "o_svpt",
    "o_1stIn",
    "o_1stWon",
    "o_2ndWon",
    "o_SvGms",
    "o_bpSaved",
    "o_bpFaced",  # opponent stats
    "minutes",  # match info
]

cols_to_sum = [
    "minutes",  # match info
    "results",  # match wins
]

In [22]:
player_stats_hist_df = add_rolling_stats(
    player_stats_hist_df, stats_columns=cols_to_mean, agg_type="mean", window=5
)

player_stats_hist_df = add_rolling_stats(
    player_stats_hist_df, stats_columns=cols_to_sum, agg_type="sum", window=5
)

In [23]:
player_stats_hist_df.shape

(196710, 73)

In [24]:
player_stats_hist_df.head(10)

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,tourney_year,match_num,player_id,player_seed,player_entry,player_name,player_hand,player_ht,player_ioc,player_age,player_rank,player_rank_points,p_ace,p_df,p_svpt,p_1stIn,p_1stWon,p_2ndWon,p_SvGms,p_bpSaved,p_bpFaced,opponent_id,opponent_seed,opponent_entry,opponent_name,opponent_hand,opponent_ht,opponent_ioc,opponent_age,opponent_rank,opponent_rank_points,o_ace,o_df,o_svpt,o_1stIn,o_1stWon,o_2ndWon,o_SvGms,o_bpSaved,o_bpFaced,score,best_of,round,minutes,results,p_ace_mean_last5,p_df_mean_last5,p_svpt_mean_last5,p_1stIn_mean_last5,p_1stWon_mean_last5,p_2ndWon_mean_last5,p_SvGms_mean_last5,p_bpSaved_mean_last5,p_bpFaced_mean_last5,opponent_rank_points_mean_last5,o_ace_mean_last5,o_df_mean_last5,o_svpt_mean_last5,o_1stIn_mean_last5,o_1stWon_mean_last5,o_2ndWon_mean_last5,o_SvGms_mean_last5,o_bpSaved_mean_last5,o_bpFaced_mean_last5,minutes_mean_last5,minutes_sum_last5,results_sum_last5
0,1992-409,Atlanta,Clay,32.0,A,1992-04-27,1992,12,100282,NaN,WC,Guillermo Vilas,L,180.0,ARG,39.6,444.0,38.0,0.0,2.0,111.0,94.0,55.0,9.0,17.0,2.0,8.0,101334,3.0,NaN,Alexander Volkov,L,188.0,RUS,25.1,18.0,1247.0,2.0,3.0,126.0,71.0,51.0,20.0,16.0,10.0,15.0,5-7 7-6(6) 6-3,3,R32,167.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1992-323,Bordeaux,Clay,32.0,A,1992-09-14,1992,7,100282,NaN,WC,Guillermo Vilas,L,180.0,ARG,40.0,410.0,47.0,0.0,3.0,81.0,41.0,26.0,20.0,13.0,5.0,9.0,102001,NaN,NaN,German Lopez,R,193.0,ESP,20.7,99.0,425.0,11.0,2.0,79.0,46.0,35.0,17.0,12.0,5.0,6.0,6-4 3-6 6-0,3,R32,109.0,0,0.0,2.00,111.000000,94.000000,55.000000,9.00,17.000000,2.0,8.000000,1247.000000,2.00,3.0,126.000000,71.000000,51.000000,20.000000,16.000000,10.00,15.000000,167.000000,167.0,0.0
2,1991-411,Chicago,Carpet,32.0,A,1991-02-25,1991,10,100284,NaN,WC,Jimmy Connors,L,178.0,USA,38.4,990.0,2.0,0.0,1.0,41.0,33.0,13.0,3.0,7.0,2.0,7.0,101409,NaN,NaN,Jaime Yzaga,R,170.0,PER,23.3,65.0,502.0,0.0,2.0,49.0,33.0,24.0,7.0,8.0,1.0,2.0,6-3 6-0,3,R32,58.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1991-403,Miami Masters,Hard,96.0,M,1991-03-15,1991,15,100284,NaN,WC,Jimmy Connors,L,178.0,USA,38.5,961.0,3.0,0.0,2.0,69.0,47.0,27.0,10.0,10.0,5.0,9.0,101274,NaN,NaN,Udo Riglewski,R,185.0,GER,24.6,98.0,364.0,3.0,4.0,70.0,33.0,21.0,13.0,10.0,5.0,11.0,6-4 6-4,3,R128,107.0,1,0.0,1.00,41.000000,33.000000,13.000000,3.00,7.000000,2.0,7.000000,502.000000,0.00,2.0,49.000000,33.000000,24.000000,7.000000,8.000000,1.00,2.000000,58.000000,58.0,0.0
4,1991-403,Miami Masters,Hard,96.0,M,1991-03-15,1991,47,100284,NaN,WC,Jimmy Connors,L,178.0,USA,38.5,961.0,3.0,0.0,3.0,50.0,33.0,15.0,8.0,9.0,2.0,7.0,101750,28.0,NaN,Cristiano Caratti,R,178.0,ITA,20.7,37.0,767.0,3.0,3.0,66.0,43.0,25.0,12.0,10.0,3.0,6.0,6-4 6-3,3,R64,80.0,0,0.0,1.50,55.000000,40.000000,20.000000,6.50,8.500000,3.5,8.000000,433.000000,1.50,3.0,59.500000,33.000000,22.500000,10.000000,9.000000,3.00,6.500000,82.500000,165.0,1.0
5,1991-409,Orlando,Hard,32.0,A,1991-04-01,1991,12,100284,NaN,WC,Jimmy Connors,L,178.0,USA,38.5,590.0,15.0,0.0,1.0,55.0,36.0,21.0,8.0,8.0,5.0,9.0,101196,4.0,NaN,Derrick Rostagno,R,185.0,USA,25.4,27.0,923.0,5.0,1.0,48.0,26.0,20.0,14.0,9.0,0.0,1.0,6-1 6-4,3,R32,71.0,0,0.0,2.00,53.333333,37.666667,18.333333,7.00,8.666667,3.0,7.666667,544.333333,2.00,3.0,61.666667,36.333333,23.333333,10.666667,9.333333,3.00,6.333333,81.666667,245.0,1.0
6,1991-329,Tokyo Outdoor,Hard,56.0,A,1991-04-08,1991,2,100284,NaN,WC,Jimmy Connors,L,178.0,USA,38.5,577.0,16.0,1.0,4.0,83.0,54.0,30.0,13.0,12.0,6.0,11.0,101004,NaN,Q,Jim Pugh,R,193.0,USA,27.1,158.0,191.0,5.0,3.0,68.0,32.0,20.0,15.0,11.0,7.0,13.0,1-6 6-4 6-0,3,R64,115.0,1,0.0,1.75,53.750000,37.250000,19.000000,7.25,8.500000,3.5,8.000000,639.000000,2.75,2.5,58.250000,33.750000,22.500000,11.500000,9.250000,2.25,5.000000,79.000000,316.0,1.0
7,1991-329,Tokyo Outdoor,Hard,56.0,A,1991-04-08,1991,26,100284,NaN,WC,Jimmy Connors,L,178.0,USA,38.5,577

In [25]:
# Save player_stats_hist_df info DataFrame as pickle
player_stats_hist_df.to_pickle("../data/player_stats_hist.pkl")

In [26]:
# read the player_stats_hist_df info from the pickle file
player_stats_hist_df = pd.read_pickle("../data/player_stats_hist.pkl")
print("Dataframe shape: ", player_stats_hist_df.shape)
player_stats_hist_df.head()

Dataframe shape:  (196710, 73)


,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,tourney_year,match_num,player_id,player_seed,player_entry,player_name,player_hand,player_ht,player_ioc,player_age,player_rank,player_rank_points,p_ace,p_df,p_svpt,p_1stIn,p_1stWon,p_2ndWon,p_SvGms,p_bpSaved,p_bpFaced,opponent_id,opponent_seed,opponent_entry,opponent_name,opponent_hand,opponent_ht,opponent_ioc,opponent_age,opponent_rank,opponent_rank_points,o_ace,o_df,o_svpt,o_1stIn,o_1stWon,o_2ndWon,o_SvGms,o_bpSaved,o_bpFaced,score,best_of,round,minutes,results,p_ace_mean_last5,p_df_mean_last5,p_svpt_mean_last5,p_1stIn_mean_last5,p_1stWon_mean_last5,p_2ndWon_mean_last5,p_SvGms_mean_last5,p_bpSaved_mean_last5,p_bpFaced_mean_last5,opponent_rank_points_mean_last5,o_ace_mean_last5,o_df_mean_last5,o_svpt_mean_last5,o_1stIn_mean_last5,o_1stWon_mean_last5,o_2ndWon_mean_last5,o_SvGms_mean_last5,o_bpSaved_mean_last5,o_bpFaced_mean_last5,minutes_mean_last5,minutes_sum_last5,results_sum_last5
0,1992-409,Atlanta,Clay,32.0,A,1992-04-27,1992,12,100282,NaN,WC,Guillermo Vilas,L,180.0,ARG,39.6,444.0,38.0,0.0,2.0,111.0,94.0,55.0,9.0,17.0,2.0,8.0,101334,3.0,NaN,Alexander Volkov,L,188.0,RUS,25.1,18.0,1247.0,2.0,3.0,126.0,71.0,51.0,20.0,16.0,10.0,15.0,5-7 7-6(6) 6-3,3,R32,167.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1992-323,Bordeaux,Clay,32.0,A,1992-09-14,1992,7,100282,NaN,WC,Guillermo Vilas,L,180.0,ARG,40.0,410.0,47.0,0.0,3.0,81.0,41.0,26.0,20.0,13.0,5.0,9.0,102001,NaN,NaN,German Lopez,R,193.0,ESP,20.7,99.0,425.0,11.0,2.0,79.0,46.0,35.0,17.0,12.0,5.0,6.0,6-4 3-6 6-0,3,R32,109.0,0,0.0,2.0,111.0,94.0,55.0,9.0,17.0,2.0,8.0,1247.0,2.0,3.0,126.0,71.0,51.0,20.0,16.0,10.0,15.0,167.0,167.0,0.0
2,1991-411,Chicago,Carpet,32.0,A,1991-02-25,1991,10,100284,NaN,WC,Jimmy Connors,L,178.0,USA,38.4,990.0,2.0,0.0,1.0,41.0,33.0,13.0,3.0,7.0,2.0,7.0,101409,NaN,NaN,Jaime Yzaga,R,170.0,PER,23.3,65.0,502.0,0.0,2.0,49.0,33.0,24.0,7.0,8.0,1.0,2.0,6-3 6-0,3,R32,58.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1991-403,Miami Masters,Hard,96.0,M,1991-03-15,1991,15,100284,NaN,WC,Jimmy Connors,L,178.0,USA,38.5,961.0,3.0,0.0,2.0,69.0,47.0,27.0,10.0,10.0,5.0,9.0,101274,NaN,NaN,Udo Riglewski,R,185.0,GER,24.6,98.0,364.0,3.0,4.0,70.0,33.0,21.0,13.0,10.0,5.0,11.0,6-4 6-4,3,R128,107.0,1,0.0,1.0,41.0,33.0,13.0,3.0,7.0,2.0,7.0,502.0,0.0,2.0,49.0,33.0,24.0,7.0,8.0,1.0,2.0,58.0,58.0,0.0
4,1991-403,Miami Masters,Hard,96.0,M,1991-03-15,1991,47,100284,NaN,WC,Jimmy Connors,L,178.0,USA,38.5,961.0,3.0,0.0,3.0,50.0,33.0,15.0,8.0,9.0,2.0,7.0,101750,28.0,NaN,Cristiano Caratti,R,178.0,ITA,20.7,37.0,767.0,3.0,3.0,66.0,43.0,25.0,12.0,10.0,3.0,6.0,6-4 6-3,3,R64,80.0,0,0.0,1.5,55.0,40.0,20.0,6.5,8.5,3.5,8.0,433.0,1.5,3.0,59.5,33.0,22.5,10.0,9.0,3.0,6.5,82.5,165.0,1.0


### Todo on player stats
todo:
1. surface stats
2. elo
3. other interesting stats features?
4. create also most recent stats file per player (tp be used for inference!!)

## Tournaments File

In [27]:
atp_matches_df.columns

Index(['tourney_id', 'tourney_name', 'surface', 'draw_size', 'tourney_level',
       'tourney_date', 'match_num', 'winner_id', 'winner_seed', 'winner_entry',
       'winner_name', 'winner_hand', 'winner_ht', 'winner_ioc', 'winner_age',
       'loser_id', 'loser_seed', 'loser_entry', 'loser_name', 'loser_hand',
       'loser_ht', 'loser_ioc', 'loser_age', 'score', 'best_of', 'round',
       'minutes', 'w_ace', 'w_df', 'w_svpt', 'w_1stIn', 'w_1stWon', 'w_2ndWon',
       'w_SvGms', 'w_bpSaved', 'w_bpFaced', 'l_ace', 'l_df', 'l_svpt',
       'l_1stIn', 'l_1stWon', 'l_2ndWon', 'l_SvGms', 'l_bpSaved', 'l_bpFaced',
       'winner_rank', 'winner_rank_points', 'loser_rank', 'loser_rank_points',
       'tourney_year'],
      dtype='object')

In [28]:
tournament_info_df = atp_matches_df[
    [
        "tourney_id",
        "tourney_name",
        "surface",
        "draw_size",
        "tourney_level",
        "tourney_date",
        "tourney_year",
    ]
].copy()
tournament_info_df = tournament_info_df.drop_duplicates().reset_index(drop=True)
tournament_info_df.shape

(2396, 7)

In [29]:
tournament_info_df.head()

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,tourney_year
0,1991-301,Auckland,Hard,32.0,A,1991-01-07,1991
1,1991-338,Sydney Outdoor,Hard,32.0,A,1991-01-07,1991
2,1991-580,Australian Open,Hard,128.0,G,1991-01-14,1991
3,1991-201,Guaruja,Hard,32.0,A,1991-02-04,1991
4,1991-408,Milan,Carpet,32.0,A,1991-02-04,1991


In [30]:
assert tournament_info_df.tourney_id.nunique() == tournament_info_df.shape[0], (
    "tourney_id should be unique in tournament_info_df"
)

In [31]:
# write the tournament info to a pickle file
tournament_info_df.to_pickle("../data/tournament_info.pkl")

In [32]:
# read the tournament info from the pickle file
tournament_info_df = pd.read_pickle("../data/tournament_info.pkl")
print("Dataframe shape: ", tournament_info_df.shape)
tournament_info_df.head()

Dataframe shape:  (2396, 7)


,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,tourney_year
0,1991-301,Auckland,Hard,32.0,A,1991-01-07,1991
1,1991-338,Sydney Outdoor,Hard,32.0,A,1991-01-07,1991
2,1991-580,Australian Open,Hard,128.0,G,1991-01-14,1991
3,1991-201,Guaruja,Hard,32.0,A,1991-02-04,1991
4,1991-408,Milan,Carpet,32.0,A,1991-02-04,1991


## ML Dataset (Training, Val and Test)